## POS tagging using modified Viterbi

### Data Preparation

In [428]:
#Importing libraries
import nltk
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import time
from nltk.tag import DefaultTagger  
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger 
from nltk.tag import TrigramTagger 
from nltk.util import ngrams
pd.set_option('display.max_rows',1000)

In [429]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [430]:
# reading the sample file
sample_file = pd.read_csv('https://cdn.upgrad.com/UpGrad/temp/9dca5f3b-53c3-47e1-86d5-5ec5dafad6f0/Test_sentences.txt',names=['sent'])

In [431]:
#splitting into train and test dataset
train,test = train_test_split(nltk_data,train_size=0.95,random_state=100)

### Build the vanilla Viterbi based POS tagger

In [432]:
train_word_set = [tup for ls in train for tup in ls ]

In [433]:
test_word_set = [tup for ls in test for tup in ls]

In [434]:
test_df = pd.DataFrame(test_word_set,columns=['Word','Tag'])

In [435]:
word_df = pd.DataFrame(train_word_set,columns=['Word','Tag'])

In [543]:
def get_accuracy(pred_df,test=test_df):
    """
    Function to calculate the accuracy of tagging the test dataset
    input params
    pred_df : dataframe containg the words and predicted tag
    test : dataframe containing words and original tags
    
    returns
    dataframe containing words wrongly tagged
    """
    print(len(pred_df),len(test))
    test['Tag_pred'] = pred_df['Tag']
    df_same = test[test['Tag']==test['Tag_pred']]
    print('Number of words in test : ',len(test),' Number of words correctly predicted : ',len(df_same))
    print('Accuracy : ',100*len(df_same)/len(test))
    return  test[test['Tag']!=test['Tag_pred']]

In [544]:
def word_given_tag(word,tag,train_set=word_df):
    tag_df = train_set.groupby(['Word','Tag']).count().reset_index()
    word_present = False
    if len(train_set[train_set['Word']==word])>0:
        word_present = True
    word_given_tag = tag_df[tag_df['Word']==word]
    
    return len(word_given_tag),len(tag_df),word_present

def tag_given_tag(tag,train_set=word_df):
    """
    function for calculating the transition probability between two tags
    input params
    tag : list containing the two tags in order
    
    returns
    counts of occurences of tag1 followed by tag2 and occurence of tag1
    """
    tag2_df = train_set[train_set['Tag']==tag[1]]
    tag1_df = train_set[train_set['Tag']==tag[0]]
    tag_1_index = np.array(tag1_df.index)+1
    tag1_by_2 = tag2_df.loc[tag2_df.index.isin(tag_1_index),:]
    return (len(tag1_by_2),len(tag1_df))

#### Calculating the transition probabilities for all tag combinations and storing in dataframe tags_df

In [484]:
T = word_df['Tag'].unique()
V = word_df['Word'].unique()
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = tag_given_tag([t1, t2])[0]/tag_given_tag([t1, t2])[1]
        
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [545]:
def Viterbi(words, tags_df=tags_df,train_bag=word_df,treat_unknown=False,treatment_func=None):
    """
    Implementation of viterbi algorithm
    input params
    words : input set of words for which tags have to be predicted
    tags_df : dataframe of transition probabilities
    train_bag : training dataframe containing word-tag pairs
    treat_unknown : whether to treat unknown words seperately; default value False
    treatment_func : name of function to treat the unknown words, is required when treat_unknown is True
    
    returns
    list of word and predicted tag pairs
    list of whether word was in training set or not, denoted by True and False respectively
    """
    V = set(words)
    v = len(V)
    word_set = list(train_bag.Word.unique())
    state = []
    presence = []
    Word_count = train_bag.reset_index().groupby(['Word','Tag'])['index'].count().reset_index().rename(
        columns={'index':'Count_Word'})
    Tag_count = train_bag.reset_index().groupby(['Tag'])['index'].count().reset_index().rename(
        columns={'index':'Count_Tag'})
    word_tag_merged = Word_count.merge(Tag_count,on='Tag',how='left')
    word_tag_merged['Emission_P'] = word_tag_merged['Count_Word']/word_tag_merged['Count_Tag']
    word_tag_pivot = pd.pivot_table(word_tag_merged,index='Word',columns='Tag',values='Emission_P').fillna(0)
    T = list(word_tag_pivot.loc['.',:].index)
    t = len(T)
    tags_df_2 = tags_df[T]
    for key, word in enumerate(words):
        is_present = True
        if word in word_set:
            emission_p = np.array(word_tag_pivot.loc[word,:])
        else:
            emission_p = np.array([0]*t)
            is_present = False
        if key==0:
            transition_p = np.array(tags_df_2.loc['.',:])
        else:
            transition_p = np.array(tags_df_2.loc[state[-1],:])
        state_prob = list(np.multiply(transition_p,emission_p))
        pmax = max(state_prob)
        state_max = T[state_prob.index(pmax)] 
        state.append(state_max)
        presence.append(is_present)

    if treat_unknown==True:
        _,state = treatment_func(state,words,presence)
    return list(zip(words, state)),presence

### Solve the problem of unknown words

## Approach 1
In the approach 1 we take a probabilistic path to determine the tags of each unknown word. We neglect the emission probabilities as they are all 0.
Instead for each missing word we consider 3 cases to calculate the transition probability:
1. tag_1, tag_missing, tag_2 : when both succeeding and preceeding words are known we calculate the probability of a particular tag T0 being the tag_missing as
               p(T0) = number of occurences of sequence (tag1,T0, tag2)/number of occurences of (tag1,T,tag2) 
       where T is the set of all available tags
2. tag_1, tag_missing : when only preceeding tag is known the expression is same as transition probability,
                p(T0) = number of occurences of sequence (tag1, T0)/number of occurence of (tag1, T)
3. tag_missing, tag_2 : when only succeeding tag is known, then we follow the same logic
                p(T0) = number of occurences of sequence (T0,tag2)/number of occurences if (T,tag2)
                
For each case we choose the tag giving maximum probability

In [546]:
def t_given_t_one_off(tag,train_set=word_df):
    """
    function for calculating probability of case 1
    input param
    tag : list of 3 tags in order
    
    returns
    counts as mentioned in above formula for case 1
    """
    tag2_df = train_set[train_set['Tag']==tag[1]]
    tag1_df = train_set[train_set['Tag']==tag[0]]
    tag3_df = train_set[train_set['Tag']==tag[2]]
    tag_1_index = np.array(tag1_df.index)+2
    tag1_by_3 = tag3_df.loc[tag3_df.index.isin(tag_1_index),:]
    tag_1_by_3_index = np.array(tag1_by_3.index)-1
    tag2_bet_1_3 = tag2_df.loc[tag2_df.index.isin(tag_1_by_3_index),:]
    return len(tag2_bet_1_3),len(tag1_by_3)

In [555]:
def approach_1(pos,words,presence):
    print('Starting approach 1')
    T = list( word_df['Tag'].unique())
    df = pd.DataFrame({'Tag':pos,'Word':words,'Present':presence})
    index = list(df[df['Present']==False].index)
    call_dict = {1:tag_given_tag,2:t_given_t_one_off}
    for i in index:
        index_prev = 0
        index_fol = len(words)-1
        if i!=0 and i!=len(words)-1:
            index_prev = i-1
            index_fol = i+1
        prev_pr = int(df.loc[index_prev,'Present'])
        fol_pr = int(df.loc[index_fol,'Present'])
        is_present = prev_pr + fol_pr
        tag = []
        if is_present==1:
            if prev_pr==1:
                tag = ['',df.loc[index_fol,'Tag']]
                pos_ch = 0
            elif fol_pr==1:
                tag = [df.loc[index_prev,'Tag'],'']
                pos_ch = 1
        elif is_present==2:
            tag = [df.loc[index_prev,'Tag'],'',df.loc[index_fol,'Tag']]
            pos_ch = 1
        else:
            continue
        prob_mat = []
        for t in T:
            tag[pos_ch] = t
            count_1,count_2 = call_dict[is_present](tag)
            prob_mat.append(count_1/count_2)
        pmax = max(prob_mat)
        state_max = T[prob_mat.index(pmax)]
        df.loc[i,'Tag'] = state_max
    print('Finished')
    return list(df['Word']),list(df['Tag'])
           

## Approach 2
For approch 2 we use lexicon based tagging utilising the default, unigram and bigram taggers in NLTK.

To model the unknown words we first replace all tags having frquency of occurence 1 in the training set with dummy word 'UNK'. This trains the model to handle unknown words as single word and assign them tags based on their preceeding words.

In [547]:
word_freq = word_df.Word.value_counts().reset_index()
low_freq_word = list(word_freq[word_freq['Word']<2]['index'])
word_df_2 = word_df.copy()
word_df_2['Word'] =  word_df_2.apply(lambda x: 'UNK' if x['Word'] in low_freq_word else x['Word'],axis=1)

In [557]:
def approach_2(pos,words,presence,train_word=word_df_2):
    print('Starting Approach 2')
    df = pd.DataFrame({'Tag':pos,'Word':words,'Present':presence})
    df['Word'] = df.apply(lambda x: 'UNK' if ~x['Present'] else x['Word'],axis=1)
    index = list(df[df['Present']==False].index)
    train_word = [[tuple(ls) for ls in word_df_2.values]]
    t0 = nltk.DefaultTagger('NOUN')
    t2 = nltk.BigramTagger(train_word, backoff=t0)
    t3 = nltk.TrigramTagger(train_word,backoff=t2)
    tagged_words = t3.tag(words)
    df_tagged = pd.DataFrame(tagged_words,columns=['Word','Tag'])
    for i in index:
        df.loc[i,'Tag'] = df_tagged.loc[i,'Tag']
    df.loc[:,'Word'] = words
    print('Finished')
    return  list(df['Word']),list(df['Tag'])

In [558]:
tagged_seq,presence = Viterbi(list(test_df['Word']))

In [559]:
tagged_seq_2,presence = Viterbi(list(test_df['Word']),treat_unknown=True,treatment_func=approach_1)

Starting approach 1
Finished


In [550]:
tagged_seq_3,presence =  Viterbi(list(test_df['Word']),treat_unknown=True,treatment_func=approach_2)

Starting Approach 2


#### Evaluating tagging accuracy

In [507]:
pred_df = pd.DataFrame(tagged_seq,columns=['Word','Tag'])
pred_df_2 = pd.DataFrame(tagged_seq_2,columns=['Word','Tag'])
pred_df_3 = pd.DataFrame(tagged_seq_3,columns=['Word','Tag'])

In [552]:
tagged = get_accuracy(pred_df)

4727 4727
Number of words in test :  4727  Number of words correctly predicted :  4268
Accuracy :  90.2898244129469


In [553]:
tagged_1 = get_accuracy(pred_df_2)

4727 4727
Number of words in test :  4727  Number of words correctly predicted :  4427
Accuracy :  93.65348000846203


In [551]:
tagged_2 = get_accuracy(pred_df_3)

4727 4727
Number of words in test :  4727  Number of words correctly predicted :  4419
Accuracy :  93.48423947535434


### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

As we see above by treating only the unknown words seperately we get an increase accuracy of more than 3% with both our approaches.

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [531]:
tagged_vit = []
tagged_app_1 = []
tagged_app_2 = []
for i,row in sample_file.iterrows():
    word = word_tokenize(row['sent'])
    tagged_seq_sample = Viterbi(word)
    tagged_seq_sample_1 = Viterbi(word,treat_unknown=True,treatment_func=approach_1)
    tagged_seq_sample_2 =  Viterbi(word,treat_unknown=True,treatment_func=approach_2)
    tagged_vit.append(tagged_seq_sample)
    tagged_app_1.append(tagged_seq_sample_1)
    tagged_app_2.append(tagged_seq_sample_2)


Starting approach 1
Starting Approach 2
Starting approach 1
Starting Approach 2
Starting approach 1
Starting Approach 2
Starting approach 1
Starting Approach 2
Starting approach 1
Starting Approach 2
Starting approach 1
Starting Approach 2
Starting approach 1
Starting Approach 2
Starting approach 1
Starting Approach 2
Starting approach 1
Starting Approach 2
Starting approach 1
Starting Approach 2
Starting approach 1
Starting Approach 2


In [538]:
word_tag_vit = [tup for ls in tagged_vit for tup in ls[0]]
presence_vit = [tup for ls in tagged_vit for tup in ls[1]]
word_tag_app_1 = [tup[1] for ls in tagged_app_1 for tup in ls[0]]
word_tag_app_2 = [tup[1] for ls in tagged_app_2 for tup in ls[0]]


In [539]:
all_words = pd.DataFrame(word_tag_vit,columns=['Word','Tag_Vit'])
all_words['Tag_Approach_1'] = word_tag_app_1
all_words['Tag_Approach_2'] = word_tag_app_2
all_words['Presence'] = presence_vit

In [556]:
print(all_words[all_words['Presence']==False].head(5))

         Word Tag_Vit Tag_Approach_1 Tag_Approach_2  Presence
0     Android       .           NOUN           NOUN     False
8      Google       .           NOUN           NOUN     False
10    Android       .           NOUN           NOUN     False
15         OS       .           NOUN           NOUN     False
16  worldwide       .           NOUN           NOUN     False


Above we see 4 instances in which both approaches correctly identifies the pos tag but the original function fails to.